In [26]:
import pandas as pd
import math
import sqlite3
import json

In [43]:
df = pd.read_excel("./Jun23.xlsx", sheet_name=" Cota vs. Área")

In [44]:
print(df.head())

       JUL2015 Unnamed: 1      JUL2019 Unnamed: 3      DIC2019 Unnamed: 5  \
0  Cota (msnm)  Área (m2)  Cota (msnm)  Área (m2)  Cota (msnm)  Área (m2)   
1      1295.04          0       1296.2          0         1292          0   
2     1295.041          0      1296.21          0      1292.01          0   
3     1295.042          0      1296.22          0      1292.02          0   
4     1295.043          0      1296.23          0      1292.03          0   

       NOV2020 Unnamed: 7      JUN2023 Unnamed: 9  
0  Cota (msnm)  Área (m2)  Cota (msnm)  Área (m2)  
1         1295          0       1292.7          0  
2      1295.01          0      1292.71        0.1  
3      1295.02          0      1292.72        0.4  
4      1295.03          0      1292.73        0.7  


In [45]:
dict_list = []
dict_list_maxmin = []

# Obtener el nombre de las columnas del DataFrame
columnas = df.columns

# Iterar sobre las columnas en pares (nombre, sin nombre)
for i in range(0, len(columnas), 2):
    # Obtener el nombre de la columna y su pareja sin nombre
    nombre_columna = columnas[i]
    sin_nombre_columna = columnas[i + 1]
    
    maximo_columna = 0
    minimo_columna = 0
    numero = 0
    # Crear un diccionario para almacenar los valores
    temp_dict = {}

    # Iterar sobre las filas para extraer los valores
    for idx, fila in df.iloc[1:].iterrows():
        # Agregar el valor al diccionario con la clave de la columna con nombre
        if pd.notna(fila[nombre_columna]):
            numero = math.ceil(float(fila[nombre_columna]) * 1000) / 1000
            if (idx == 1):
                maximo_columna = numero
                minimo_columna = numero
            else:
                if numero > maximo_columna:
                    maximo_columna = numero
                if numero < minimo_columna:
                    minimo_columna = numero
            temp_dict[numero] = fila[sin_nombre_columna]
    
    # Agregar el diccionario a la lista
    dict_list.append(temp_dict)
    temp_dict_maxmin = {"max": maximo_columna, "min": minimo_columna}
    dict_list_maxmin.append(temp_dict_maxmin)

# Mostrar el contenido de la lista de diccionarios
for i, d in enumerate(dict_list, 1):
    print(f'Diccionario {i}: {d}')

Diccionario 1: {1295.04: 0, 1295.041: 0, 1295.042: 0, 1295.043: 0, 1295.044: 0, 1295.045: 0, 1295.046: 0, 1295.047: 0, 1295.048: 0.1, 1295.049: 0.1, 1295.05: 0.1, 1295.051: 0.1, 1295.052: 0.1, 1295.053: 0.1, 1295.054: 0.1, 1295.055: 0.1, 1295.056: 0.1, 1295.057: 0.1, 1295.058: 0.1, 1295.059: 0.1, 1295.06: 0.1, 1295.061: 0.1, 1295.062: 0.1, 1295.063: 0.1, 1295.064: 0.1, 1295.065: 0.1, 1295.066: 0.1, 1295.067: 0.1, 1295.068: 0.1, 1295.069: 0.1, 1295.07: 0.1, 1295.071: 0.1, 1295.072: 0.1, 1295.073: 0.1, 1295.074: 0.1, 1295.075: 0.1, 1295.076: 0.1, 1295.077: 0.1, 1295.078: 0.1, 1295.079: 0.1, 1295.08: 0.2, 1295.081: 0.2, 1295.082: 0.2, 1295.083: 0.2, 1295.084: 0.2, 1295.085: 0.2, 1295.086: 0.2, 1295.087: 0.2, 1295.088: 0.2, 1295.089: 0.2, 1295.09: 0.2, 1295.091: 0.2, 1295.092: 0.2, 1295.093: 0.2, 1295.094: 0.2, 1295.095: 0.2, 1295.096: 0.2, 1295.097: 0.2, 1295.098: 0.2, 1295.099: 0.2, 1295.1: 0.2, 1295.101: 0.2, 1295.102: 0.3, 1295.103: 0.3, 1295.104: 0.3, 1295.105: 0.3, 1295.106: 0.3, 129

In [46]:
print(dict_list_maxmin)

[{'max': 1307.496, 'min': 1295.04}, {'max': 1307.5, 'min': 1296.2}, {'max': 1307.5, 'min': 1292.0}, {'max': 1307.5, 'min': 1295.0}, {'max': 1323.6, 'min': 1292.7}]


In [51]:
# Conectar a la base de datos SQLite3
conexion = sqlite3.connect('../db.sqlite3')

# Crear un cursor
cursor = conexion.cursor()

# Definir la sentencia SQL de inserción
sql_insert = '''INSERT INTO recursos_mediciones (tipo, fecha, json, id_embalse_id, id_empresa_id, max, min)
                VALUES (?, ?, ?, ?, ?, ?, ?)'''
num = 4
fecha = ['Julio 2015', 'Julio 2019', 'Diciembre 2019', 'Noviembre 2020','Junio 2023']
# Valores a insertar en la tabla
valores = ('Cota vs. Área', fecha[num], json.dumps(dict_list[num]), 1, 1, dict_list_maxmin[num]["max"], dict_list_maxmin[num]["min"])

# Ejecutar la sentencia SQL de inserción con los valores
cursor.execute(sql_insert, valores)

# Confirmar los cambios
conexion.commit()

# Cerrar la conexión
conexion.close()